In [50]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [51]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df_nb_id = application_df.drop(["EIN", "NAME"], axis = 1)
df_nb_id.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [52]:
# Determine the number of unique values in each column.
def unique_values_in_columns(df_nb_id):
    unique_counts = df_nb_id.nunique()
    return unique_counts

result = unique_values_in_columns(df_nb_id)
print(result)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [53]:
# Look at APPLICATION_TYPE value counts for binning
df_nb_id["APPLICATION_TYPE"].value_counts()


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [54]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = df_nb_id["APPLICATION_TYPE"].value_counts().loc[df_nb_id["APPLICATION_TYPE"].value_counts()<725].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df_nb_id["APPLICATION_TYPE"] = df_nb_id["APPLICATION_TYPE"].replace(app,"Other")

# Check to make sure binning was successful
df_nb_id["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [55]:
# You may find it helpful to look at CLASSIFICATION value counts >1
df_nb_id["CLASSIFICATION"].value_counts().loc[df_nb_id["CLASSIFICATION"].value_counts()>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [56]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = df_nb_id["CLASSIFICATION"].value_counts().loc[df_nb_id["CLASSIFICATION"].value_counts()<778].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [57]:
# Convert categorical data to numeric with `pd.get_dummies`
df_numerical = pd.get_dummies(df_nb_id)
df_numerical.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [58]:
# Split our preprocessed data into our features and target arrays
y = df_numerical["IS_SUCCESSFUL"]
X = df_numerical.drop(columns="IS_SUCCESSFUL")
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)

In [59]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential([
  tf.keras.layers.Dense(units=30,activation="relu",input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(units=10,activation="relu"),
  tf.keras.layers.Dense(units=1,activation="sigmoid")
])


# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 30)                3240      
                                                                 
 dense_10 (Dense)            (None, 10)                310       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 3561 (13.91 KB)
Trainable params: 3561 (13.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
# Checkpoint dependencies import
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [62]:
# Compile the model
nn.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])

In [63]:
# This callback saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])


Epoch 1/50
799/804 [============================>.] - ETA: 0s - loss: 0.5825 - accuracy: 0.7139
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 4s 3ms/step - loss: 0.5824 - accuracy: 0.7140
Epoch 2/50
790/804 [============================>.] - ETA: 0s - loss: 0.5544 - accuracy: 0.7302
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7305
Epoch 3/50
798/804 [============================>.] - ETA: 0s - loss: 0.5497 - accuracy: 0.7312
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7314
Epoch 4/50
803/804 [============================>.] - ETA: 0s - loss: 0.5479 - accuracy: 0.7311
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7313
Epoch 5/50
804/804 [============================

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5535 - accuracy: 0.7300 - 483ms/epoch - 2ms/step
Loss: 0.553464412689209, Accuracy: 0.7300291657447815


In [66]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_2 = tf.keras.models.Sequential([
  tf.keras.layers.Dense(units=80,activation="LeakyReLU",input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(units=30,activation="LeakyReLU"),
  tf.keras.layers.Dense(units=1,activation="sigmoid")
])


# Check the structure of the model
nn_2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 80)                8640      
                                                                 
 dense_14 (Dense)            (None, 30)                2430      
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 11101 (43.36 KB)
Trainable params: 11101 (43.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
# Compile the model
nn_2.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])

In [68]:
# This callback saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn_2.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
799/804 [============================>.] - ETA: 0s - loss: 0.5819 - accuracy: 0.7232
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 0.5820 - accuracy: 0.7229
Epoch 2/50
794/804 [============================>.] - ETA: 0s - loss: 0.5573 - accuracy: 0.7297
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5573 - accuracy: 0.7297
Epoch 3/50
794/804 [============================>.] - ETA: 0s - loss: 0.5541 - accuracy: 0.7302
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7308
Epoch 4/50
791/804 [============================>.] - ETA: 0s - loss: 0.5519 - accuracy: 0.7311
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7312
Epoch 5/50
804/804 [============================

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5513 - accuracy: 0.7289 - 749ms/epoch - 3ms/step
Loss: 0.5513427257537842, Accuracy: 0.728863000869751


In [82]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_3 = tf.keras.models.Sequential([
  tf.keras.layers.Dense(units=100,activation="relu",input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(units=50,activation="relu"),
  tf.keras.layers.Dense(units=25,activation="relu"),
  tf.keras.layers.Dense(units=10,activation="relu"),
  tf.keras.layers.Dense(units=1,activation="sigmoid")
])


# Check the structure of the model
nn_3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 100)               10800     
                                                                 
 dense_29 (Dense)            (None, 50)                5050      
                                                                 
 dense_30 (Dense)            (None, 25)                1275      
                                                                 
 dense_31 (Dense)            (None, 10)                260       
                                                                 
 dense_32 (Dense)            (None, 1)                 11        
                                                                 
Total params: 17396 (67.95 KB)
Trainable params: 17396 (67.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [83]:
# Compile the model
nn_3.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])

In [84]:
# This callback saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=2,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn_3.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
794/804 [============================>.] - ETA: 0s - loss: 0.5746 - accuracy: 0.7190
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 0.5744 - accuracy: 0.7191
Epoch 2/100
799/804 [============================>.] - ETA: 0s - loss: 0.5518 - accuracy: 0.7320
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7320
Epoch 3/100
787/804 [============================>.] - ETA: 0s - loss: 0.5480 - accuracy: 0.7336
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7334
Epoch 4/100
803/804 [============================>.] - ETA: 0s - loss: 0.5461 - accuracy: 0.7337
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7336
Epoch 5/100
804/804 [=======================

In [85]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5649 - accuracy: 0.7300 - 510ms/epoch - 2ms/step
Loss: 0.5648714303970337, Accuracy: 0.7300291657447815


In [86]:
# Export our model to HDF5 file
nn_3.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
